# DIC - Assignment - 3

In [1]:
# pip install pyspark==2.4.0 findspark numpy

In [2]:
# pip list --format=columns3

In [3]:
# Imports and pyspark config. data from Demo video:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Import module
import os
import csv
import json
import re

In [4]:
# Part1DataFrame = spark.read.csv(
#     "/home/cse587/Desktop/Proj3/train.csv", 
#     header=True, schema=Proj3Part1, sep=",")
# #show 3 rows of our DataFrame
# Part1DataFrame.show(20) ##### ERROR: Not working on train.csv even with "," separator...

# read csv file with our defined schema into Spark DataFrame, and use "," delimiter
folder_path = "/home/cse587/Desktop/Proj3"
file_name = "train.csv"

#file_path = f"{folder_path}/{file_name}"
file_path = "train.csv"

file_data = []
keys_processed = []
all_genres = [] # Holds unique genre values from train file
with open(file_path,"r") as csv_file:
    csv_reader = csv.DictReader(csv_file,delimiter=",")
    
    for row in csv_reader:
        movie_id = row.get("movie_id")
        movie_id = int(movie_id) if movie_id else None
        movie_name = str(row.get("movie_name") or "").strip()
        plot = str(row.get("plot") or "").strip()
        
        genre = row.get("genre") #Parse genre into an array as final value
        
        try:
            genre = genre.replace("[", "").replace("]", "").replace("'", "").split(",")
        except Exception as err:
            print(genre, err)
        
        
        if isinstance(genre, list):
            genre_normalized = []
            for item in genre:
                val = str(item).strip().lower() if item else None
                if val:
                    genre_normalized.append(val)
                    all_genres.append(val)
            genre = genre_normalized
        elif isinstance(genre, str):
            if genre:
                val = str(genre).strip().lower()
                all_genres.append(val)
                genre = [val]
        else:
            genre = []
        
        genre.sort() # Store in sorted order
        
        
        # Clean up the plot
        plot_norm = None
        try:
            _norm = str(plot).lower().strip()
            _norm = re.sub(r"[^a-z]+", " ", _norm)            
            _arr = _norm.split(" ")
            plot_norm = " ".join(_arr).strip()
        except Exception as err:
            print(err)
        
                
        row_data = [
            movie_id,
            movie_name,
            plot,
            plot_norm,
            genre,
            
        ]
        
        movie_key = (movie_id, movie_name)
        
        if not all(row_data) or movie_key in keys_processed:
            continue
            
        #making a dict for file_data:
        keys_processed.append(movie_key)
        file_data.append({
            "movie_id": movie_id,
            "movie_name": movie_name,
            "plot": plot,
            "plot_normalized": plot_norm,
            "genre": genre,
        })

# all_genres = list(set(all_genres)) # Ensure only unique values are stored in lowercase
# all_genres.sort() # Store in sorted order

#all_genres as per mapping.csv file
all_genres = ['drama', 'comedy', 'romance film', 'thriller', 'action', 'world cinema', 'crime fiction', 'horror', 'black-and-white', 'indie', 'action/adventure', 'adventure', 'family film', 'short film', 'romantic drama', 'animation', 'musical', 'science fiction', 'mystery', 'romantic comedy']

# all_genres = list(set(all_genres))
# print(len(all_genres), all_genres)

print(json.dumps(file_data[0:1], sort_keys=True, indent=2))

[
  {
    "genre": [
      "drama",
      "world cinema"
    ],
    "movie_id": 23890098,
    "movie_name": "Taxi Blues",
    "plot": "Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.",
    "plot_normalized": "shlykov a hard working taxi driver and lyosha a saxophonist develop a bizarre love hate relationship and despite their prejudices realize they aren t so different after all"
  }
]


In [5]:
def encode_genre(movie_genre):
    encoded = ["0"] * len(all_genres)
    for item in movie_genre:
        try:
            idx = all_genres.index(item)
            encoded[idx] = "1"
        except:
            pass
    
    return "".join(encoded)
        
def decode_genre(encoded_str):
    decoded = []
    encoded = list(encoded_str)
    for idx in range(0, len(encoded)):
        if encoded[idx] != "1":
            continue
            
        val = all_genres[idx]
        decoded.append(val)
    return decoded

# Transform each movie genre into a matrix
for item in file_data:
    encoded_val = encode_genre(item['genre'])
    item['genre_encoded'] = encoded_val
    
    # Print a sammple
    #print(encoded_val, decode_genre(encoded_val))
    
#checking if encoder and decoder worked correctly
print(json.dumps(file_data[0:1], sort_keys=True, indent=2))

[
  {
    "genre": [
      "drama",
      "world cinema"
    ],
    "genre_encoded": "10000100000000000000",
    "movie_id": 23890098,
    "movie_name": "Taxi Blues",
    "plot": "Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.",
    "plot_normalized": "shlykov a hard working taxi driver and lyosha a saxophonist develop a bizarre love hate relationship and despite their prejudices realize they aren t so different after all"
  }
]


### Create session object in order to access all Spark API

In [6]:
#spark_folder = "/home/cse587/spark-2.4.0-bin-hadoop2.7"
spark_folder = "/usr/local/Cellar/apache-spark/2.4.5/libexec"

findspark.init(spark_home=spark_folder)


spark = SparkSession \
    .builder \
    .appName("Proj3Part1") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

In [7]:
#define data schema for file we want to read
df_schema = StructType([
    StructField("movie_id", IntegerType(), False),
    StructField("movie_name", StringType(), False),
    StructField("plot", StringType(), False),
    StructField("plot_normalized", StringType(), False),
    StructField("genre", StringType(), False),
    StructField("genre_encoded", StringType(),False),
    StructField("genre0", IntegerType(),False),
    StructField("genre1", IntegerType(),False),
    StructField("genre2", IntegerType(),False),
    StructField("genre3", IntegerType(),False),
    StructField("genre4", IntegerType(),False),
    StructField("genre5", IntegerType(),False),
    StructField("genre6", IntegerType(),False),
    StructField("genre7", IntegerType(),False),
    StructField("genre8", IntegerType(),False),
    StructField("genre9", IntegerType(),False),
    StructField("genre10", IntegerType(),False),
    StructField("genre11", IntegerType(),False),
    StructField("genre12", IntegerType(),False),
    StructField("genre13", IntegerType(),False),
    StructField("genre14", IntegerType(),False),
    StructField("genre15", IntegerType(),False),
    StructField("genre16", IntegerType(),False),
    StructField("genre17", IntegerType(),False),
    StructField("genre18", IntegerType(),False),
    StructField("genre19", IntegerType(),False)
])

### Counting number of rows, printing dataframe schema and printing statistic of our data

In [8]:

# Create data frame
struct_key_map = ["movie_id", "movie_name", "plot", "plot_normalized", "genre", "genre_encoded"]
spark_data = []
for items in file_data:
    data_len = len(struct_key_map)
    row = [None] * data_len
    
    for idx in range(0, data_len):
        key = struct_key_map[idx]
        row[idx] = items[key]
    
    row.extend([int(c) for c in items["genre_encoded"]])

    spark_data.append(row)
    
    

rdd = spark.sparkContext.parallelize(spark_data)

df = spark.createDataFrame(rdd, df_schema)
print(df.schema)
df.show()

StructType(List(StructField(movie_id,IntegerType,false),StructField(movie_name,StringType,false),StructField(plot,StringType,false),StructField(plot_normalized,StringType,false),StructField(genre,StringType,false),StructField(genre_encoded,StringType,false),StructField(genre0,IntegerType,false),StructField(genre1,IntegerType,false),StructField(genre2,IntegerType,false),StructField(genre3,IntegerType,false),StructField(genre4,IntegerType,false),StructField(genre5,IntegerType,false),StructField(genre6,IntegerType,false),StructField(genre7,IntegerType,false),StructField(genre8,IntegerType,false),StructField(genre9,IntegerType,false),StructField(genre10,IntegerType,false),StructField(genre11,IntegerType,false),StructField(genre12,IntegerType,false),StructField(genre13,IntegerType,false),StructField(genre14,IntegerType,false),StructField(genre15,IntegerType,false),StructField(genre16,IntegerType,false),StructField(genre17,IntegerType,false),StructField(genre18,IntegerType,false),StructField

In [9]:
#count number of rows of our dataFrame
num_rows = df.count()
print("number of rows: ", num_rows)

#show our dataFrame schema
df.printSchema()
#show statistic of the data we want
df.describe('genre').show()

number of rows:  31108
root
 |-- movie_id: integer (nullable = false)
 |-- movie_name: string (nullable = false)
 |-- plot: string (nullable = false)
 |-- plot_normalized: string (nullable = false)
 |-- genre: string (nullable = false)
 |-- genre_encoded: string (nullable = false)
 |-- genre0: integer (nullable = false)
 |-- genre1: integer (nullable = false)
 |-- genre2: integer (nullable = false)
 |-- genre3: integer (nullable = false)
 |-- genre4: integer (nullable = false)
 |-- genre5: integer (nullable = false)
 |-- genre6: integer (nullable = false)
 |-- genre7: integer (nullable = false)
 |-- genre8: integer (nullable = false)
 |-- genre9: integer (nullable = false)
 |-- genre10: integer (nullable = false)
 |-- genre11: integer (nullable = false)
 |-- genre12: integer (nullable = false)
 |-- genre13: integer (nullable = false)
 |-- genre14: integer (nullable = false)
 |-- genre15: integer (nullable = false)
 |-- genre16: integer (nullable = false)
 |-- genre17: integer (nullable

## Model

In [10]:
from pyspark.ml.feature import StopWordsRemover, Tokenizer, VectorAssembler

tokenizer = Tokenizer(inputCol='plot_normalized', outputCol="plot_normalized_temp")
plot_token_df = tokenizer.transform(df)
plot_token_df = plot_token_df.drop("plot_normalized")
plot_token_df = plot_token_df.withColumnRenamed("plot_normalized_temp", "plot_normalized")

#Removing stop-words:
remover = StopWordsRemover(inputCol='plot_normalized', outputCol='plot_normalized_temp')
plot_token_df = remover.transform(plot_token_df)
plot_token_df = plot_token_df.drop("plot_normalized")
plot_token_df = plot_token_df.withColumnRenamed("plot_normalized_temp", "plot_normalized")

In [11]:
#creating a new dataframe to train our model on:
training_df = plot_token_df

training_df = training_df.drop("plot")
training_df = training_df.withColumnRenamed("plot_normalized", "plot")

training_df.show()

+--------+--------------------+--------------------+--------------------+------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------------------+
|movie_id|          movie_name|               genre|       genre_encoded|genre0|genre1|genre2|genre3|genre4|genre5|genre6|genre7|genre8|genre9|genre10|genre11|genre12|genre13|genre14|genre15|genre16|genre17|genre18|genre19|                plot|
+--------+--------------------+--------------------+--------------------+------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------------------+
|23890098|          Taxi Blues|[drama, world cin...|10000100000000000000|     1|     0|     0|     0|     0|     1|     0|     0|     0|     0|      0|      0|      0|      0|      0|      0|      0|      0|      0|      0|[shlykov, hard, w...|
|31186339|    The Hu

### TF - IDF

In [12]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

hashingTF = HashingTF(inputCol="plot", outputCol="TF_plot", numFeatures=10000) #numFeatures = max_features in tfidf_vectorizer
featurizedData = hashingTF.transform(training_df)

# alternatively, CountVectorizer can also be used to get term frequency vectors
idf = IDF(inputCol="TF_plot", outputCol="TF_IDF_plot")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("movie_id", "TF_IDF_plot").show()

+--------+--------------------+
|movie_id|         TF_IDF_plot|
+--------+--------------------+
|23890098|(10000,[135,719,1...|
|31186339|(10000,[15,26,47,...|
|20663735|(10000,[52,74,104...|
| 2231378|(10000,[52,76,77,...|
|  595909|(10000,[15,91,137...|
| 5272176|(10000,[52,158,20...|
| 1952976|(10000,[15,104,14...|
|24225279|(10000,[17,24,76,...|
| 2462689|(10000,[1,15,67,7...|
|20532852|(10000,[364,585,8...|
|15401493|(10000,[76,100,15...|
|18188932|(10000,[76,390,46...|
| 2940516|(10000,[50,410,61...|
| 1480747|(10000,[15,52,104...|
|24448645|(10000,[87,141,81...|
|15072401|(10000,[213,493,7...|
| 4018288|(10000,[230,433,5...|
| 4596602|(10000,[164,277,3...|
|15224586|(10000,[273,306,3...|
|15585766|(10000,[87,207,21...|
+--------+--------------------+
only showing top 20 rows



### Building the actual model: Random Forest classifier

In [13]:
import collections

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier


movie_id_to_predictions = collections.defaultdict(list)
trainingData, testData = rescaledData.randomSplit([0.8, 0.2])
    
for i in range(9, 10):
    rf = RandomForestClassifier(labelCol=f"genre{i}", \
                                featuresCol="TF_IDF_plot", \
                                predictionCol=f"prediction{i}", \
                                rawPredictionCol=f"rawPrediction{i}", \
                                probabilityCol=f"probability{i}", \
                                numTrees = 40, \
                                maxDepth = 4, \
                                maxBins = 32)# Train model with Training Data
    
    rfModel = rf.fit(trainingData)
    predictions = rfModel.transform(testData)

    movie_ith_genre_prediction = predictions.rdd.map(lambda p: (p.movie_id, p[f"prediction{i}"])).collect()
        
    for movie_id, prediction in movie_ith_genre_prediction:
        movie_id_to_predictions[movie_id].append(int(prediction))
    
    evaluator = MulticlassClassificationEvaluator(labelCol=f"genre{i}", predictionCol=f"prediction{i}", metricName="accuracy")
    print(f"Accuracy for {all_genres[i]} (genre{i}) prediction: {evaluator.evaluate(predictions)}")
    
    # predictions.select("movie_name", f"genre{i}", f"prediction{i}").show()

Accuracy for indie (genre9) prediction: 0.9042145593869731


### Reason why the code is wrong:
The code snipped below doesnot print anything and hence it is proven that our code could not classify on the multi-label scale.
All it did was predict one genre at a time.

In [14]:
for key, value in movie_id_to_predictions.items():
    if sum(value) > 1:
        print(f"{key}: {value}")